In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
import cv2
import numpy as np
import os
import PIL
from tensorflow.keras.utils import to_categorical
from pathlib import Path
import zipfile
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode

In [ ]:
train_df = pd.DataFrame(columns=['directory', 'label'])

for i in os.listdir('../input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients'):
    for j in os.listdir('../input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients' + '/' + i + '/'):
        train_df = pd.concat([train_df, pd.DataFrame({'directory': ['../input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients' + '/' + i + '/' + j],
                                                     'label': [i]})], ignore_index=True)

In [ ]:
class_names={
             'NITROGEN':0,
             'PHOSPHORUS':1,
             'POTASSIUM':2
             }

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
train_df['label_encoded'] = label_encoder.transform(train_df['label'])

In [ ]:
train_df.drop(columns=['label'], inplace=True) 

In [ ]:
train_df

In [ ]:
image_list = []
label_list = []
for i in range(len(train_df)):
    image = cv2.imread(train_df['directory'][i])
    image = cv2.resize(image,(256,1280))
    image_list.append(image)
    label_list.append(train_df['label_encoded'][i])

image_list = np.array(image_list)
label_list = np.array(label_list)

In [ ]:
train_x = image_list[:1100]  
test_x = image_list[1100:] 
train_y = label_list[:1100]  
test_y = label_list[1100:]

In [ ]:
num_parts = 5
new_height = train_x[0].shape[0] // num_parts
x_train_parts = [image[i:i+new_height, :] for image in train_x for i in range(0, train_x[0].shape[0], new_height)]
y_train_parts = []
for i in range(len(train_x)):
    y_train_parts.extend([train_y[i]] * num_parts)
y_train_parts = np.array(y_train_parts)

In [ ]:
labels = [0, 1, 2]
encoded_labels = to_categorical(y_train_parts, num_classes=3)
print(encoded_labels.shape)

In [ ]:
num_classes=3
x_train_parts=np.array(x_train_parts)
y_train_encoded = tf.one_hot(y_train_parts, depth=num_classes, dtype=tf.float32)
y_train_encoded = np.array(y_train_encoded) 
x_train_parts = x_train_parts.astype('float32')
x_train_parts = x_train_parts/255

In [ ]:
x_train_parts

In [ ]:
x_train_parts.shape

In [ ]:
encoded_labels.shape

In [ ]:
train_df = pd.DataFrame(columns=['directory', 'label'])
for i in os.listdir('../input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients'):
    for j in os.listdir('../input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients' + '/' + i + '/'):
        train_df = pd.concat([train_df, pd.DataFrame({'directory': ['../input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients' + '/' + i + '/' + j],
                                                      'label': [i]})], ignore_index=True)

label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])
train_df.drop(columns=['label'], inplace=True)
image_list = [cv2.resize(cv2.imread(directory), (256, 1280)) for directory in train_df['directory']]
num_parts = 5
new_width = 256
new_height = 256
x_train_parts = []
y_train_parts = []

for image in image_list:
    for i in range(0, image.shape[1], new_width):
        for j in range(0, image.shape[0], new_height):
            x_train_parts.append(image[j:j+new_height, i:i+new_width])
x_train_parts = np.array(x_train_parts)
y_train_parts = np.repeat(train_df['label_encoded'], num_parts)

In [ ]:
x_train_parts = x_train_parts.astype('float32') / 255
encoded_labels = to_categorical(y_train_parts, num_classes=3)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train_parts, y_train_parts, test_size=0.1, random_state=42)
encoded_labels_train = to_categorical(y_train, num_classes=3)
encoded_labels_val = to_categorical(y_val, num_classes=3)

In [ ]:
model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(256, 256, 3)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, encoded_labels_train, epochs=5, validation_data=(x_val, encoded_labels_val))

In [ ]:
model.save("deciese")

In [ ]:
from keras.models import load_model

In [ ]:
loaded_model = load_model('/kaggle/working/deciese')

In [ ]:
encoded_labels_train = to_categorical(y_train, num_classes=3)
encoded_labels_val = to_categorical(y_val, num_classes=3)

In [ ]:
loss, accuracy = loaded_model.evaluate(x_val, encoded_labels_val)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
image_list=np.array(image_list)
image_list[0].shape

In [ ]:
encoded_labels_train = to_categorical(y_train, num_classes=3)
encoded_labels_val = to_categorical(y_val, num_classes=3)

In [ ]:
label_encoder = LabelEncoder()
original_labels = ['NITROGEN', 'PHOSPHORUS', 'POTASSIUM']  # Replace with your actual class labels
label_encoder.fit(original_labels)

In [ ]:
image_list[150].shape

In [ ]:
num_parts = 5
new_width = 256
new_height = 256
image_parts = []
single_image=image_list[150]
for i in range(0, single_image.shape[1], new_width):
    for j in range(0, single_image.shape[0], new_height):
        image_parts.append(single_image[j:j+new_height, i:i+new_width])

# Convert the list of image parts to a NumPy array
image_parts = np.array(image_parts)

# Normalize pixel values
image_parts = image_parts.astype('float32') / 255

# Reshape the image parts array to match the input shape expected by the model
image_parts = image_parts.reshape((-1, 256, 256, 3))
predictions = loaded_model.predict(image_parts)

# Get the class with the highest probability for each part
predicted_classes = np.argmax(predictions, axis=1)
predicted_probabilities = np.max(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

majority_class, _ = mode(predicted_labels)
average_probability = np.mean(predicted_probabilities)
print(f"Combined Predicted Label (Majority Class): {majority_class[0]}")
print(f"Combined Predicted Probability (Average): {average_probability:.4f}")
plt.imshow(single_image)

In [ ]:
# the following bellow code can not work in kaggle notebook
from flask import Flask, render_template_string, request
import cv2
import numpy as np
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

app = Flask(__name__)

# Define label encoder
original_labels = ['NITROGEN', 'PHOSPHORUS', 'POTASSIUM']
label_encoder = LabelEncoder()
label_encoder.fit(original_labels)

@app.route('/', methods=['GET', 'POST'])
def predict():
    prediction_result = None
    selected_image = None

    if request.method == 'POST':
        # Get the file from the POST request
        file = request.files['file']

        # Read the image
        image = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (256, 1280))

        num_parts = 5
        new_width = 256
        new_height = 256
        image_parts = []
        for i in range(0, image.shape[1], new_width):
            for j in range(0, image.shape[0], new_height):
                image_parts.append(image[j:j+new_height, i:i+new_width])

        # Convert the list of image parts to a NumPy array
        image_parts = np.array(image_parts)

        # Normalize pixel values
        image_parts = image_parts.astype('float32') / 255

        # Reshape the image parts array to match the input shape expected by the model
        image_parts = image_parts.reshape((-1, 256, 256, 3))

        # Assuming you have loaded your pre-trained model as 'loaded_model'
        # Load your model here

        # Make predictions
        predictions = loaded_model.predict(image_parts)

        # Get the class with the highest probability for each part
        predicted_classes = np.argmax(predictions, axis=1)
        predicted_labels = [label_mapping[p] for p in predicted_classes]

        # Calculate the average probability
        average_probability = np.mean(np.max(predictions, axis=1))
        unique_labels, counts = np.unique(predicted_labels, return_counts=True)
        max_count_index = np.argmax(counts)
        majority_class = unique_labels[max_count_index]
        prediction_result = f"Combined Predicted Label (Majority Class): {majority_class}, Combined Predicted Probability (Average): {average_probability:.4f}"

        # Convert the selected image to base64 encoding
        _, buffer = cv2.imencode('.jpg', image)
        image_encoded = base64.b64encode(buffer).decode('utf-8')
        selected_image = f"data:image/jpeg;base64,{image_encoded}"
         return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Upload Image</title>
            <style>
       
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh; /* Set height to viewport height for vertical centering */
            margin: 0; /* Remove default margin */
            padding: 0; /* Remove default padding */
        }
        .content {
            text-align: center;
        }
    </style>
        </head>
        <body>
            <h1>Upload an Image</h1>
            <form method="post" enctype="multipart/form-data">
                <input type="file" name="file" accept="image/*">
                <input type="submit" value="Upload">
            </form>
            {% if selected_image %}
            <div>
                <h2>Selected Image</h2>
                <img src="{{ selected_image }}" alt="Selected Image">
            </div>
            {% endif %}
            {% if prediction_result %}
            <div>
                <h2>Prediction Result</h2>
                <p>{{ prediction_result }}</p>
            </div>
            {% endif %}
        </body>
        </html>
    ''', prediction_result=prediction_result, selected_image=selected_image)

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)